## Imports

In [1]:
import os
import sys
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import midiutil
from scipy.io import wavfile
from scipy.signal import spectrogram, get_window
from os import listdir
from midiutil import MIDIFile

## Process

In [2]:
def stft (d,fs,winlen,nfft):
    noverlap=fs/100
    nperseg=winlen
    win = get_window('hamming',winlen)
    freqbin,tbin,spec=spectrogram(x=d,fs=fs,noverlap=noverlap,nperseg=nperseg,nfft=nfft,window=win)
    spec=(spec*1000)+1
    spec=np.real(np.log(spec))
    spec[spec<0]=0
    return spec, freqbin, tbin

In [3]:
def genData (spec,freqbin,tbin):
    Q=np.zeros([tbin.shape[0],Notes.shape[0]])
    for i in range(spec.shape[1]):
        Z=spec[:,i]
        for j in range(Notes.shape[0]-1):
            if j==0:
                j=1
            f1=Notes[j-1]
            f2=Notes[j+1]
            wb1 = np.where(f1>=freqbin)[0]
            wb2 = np.where(f2<=freqbin)[0]
            wb=np.intersect1d(wb1,wb2)
            bn=np.argmin(np.abs(freqbin-Notes[j]))
        
            if ((wb.shape[0]==0) or wb.shape[0]==1):
                Q[i,j]=Z[j]
            
            else:
                x1 = wb[0:np.where(wb==bn)[0][0]+1]
                x2 = wb[np.where(wb==bn)[0][0]:]
                y1 = (x1-wb[0])/(x1.shape[0]-1)
                y2 = 1-(x2-bn)/(x2.shape[0]-1)
                y1[np.isnan(y1)]=1
                y2[np.isnan(y2)]=1
                Q[i,j]= ((np.matmul(Z[x1],y1)+np.matmul(Z[x2],y2)-Z[bn])/(np.sum(y1)+np.sum(y2)))
    Q=Q[:,1:100]
    a=np.diff(Q)
    Q=np.concatenate((Q,a),axis=1)
    return Q

In [4]:
# Define frequencies for notes up to 100 (first harmonic of 88th note)
Notes = np.zeros(101)
Notes[0] = 0
Notes[1] = 27.5
for i in range(99):
    Notes[i+2] = Notes[i+1]*(np.power(2,1/12))

In [5]:
winlen1=2048
nfft1=2048

winlen2=8192
nfft2=8192

In [24]:
dir_path_in = os.path.join(os.getcwd(),'Data/Recordings/Sound')
out_path = os.path.join(os.getcwd(),'Data/Processed/input.txt')

out_path1 = os.path.join(os.getcwd(),'Data/Processed/output.txt')

files = listdir(dir_path_in)
inp_final = np.zeros((1,394))
out_final = np.zeros((1,88))
for i in range(len(files)):
    print(i/len(files))
    wav_path = os.path.join(os.getcwd(), 'Data/Recordings/Sound', files[i])
    fs, data = wavfile.read(wav_path)
    data = data[:,1]
    spec1,freqbin1,tbin1 = stft(d=data,fs=fs,winlen=winlen1,nfft=nfft1)
    spec2,freqbin2,tbin2 = stft(d=data,fs=fs,winlen=winlen2,nfft=nfft2)
    
    Q1_temp = genData(spec=spec1,freqbin=freqbin1,tbin=tbin1)
    Q2 = genData(spec=spec2,freqbin=freqbin2,tbin=tbin2)

    Q1 = Q2.copy()
    
    for j in range(tbin2.shape[0]):
        near_t = np.argmin(np.abs(tbin2[j]-tbin1))
        Q1[j,:]=Q1_temp[near_t,:]
    
    inp_final = np.append(inp_final,np.concatenate((Q1,Q2),axis=1),axis=0)
    
    truth_path = os.path.join(os.getcwd(), 'Data/Recordings/Truth', files[i][:-3]+'txt')
    truth_table = pd.read_csv(truth_path,sep='	',header=None)
    truth_table_clean = truth_table[1:truth_table.shape[0]].copy().get_values()
    truth_table_clean=truth_table_clean.astype(np.float)
    truth_table_clean[:,2] = truth_table_clean[:,2]-21

    truth_notes=np.zeros((tbin2.shape[0],88))
    for k in range(truth_table_clean.shape[0]):
        a=np.where(tbin2>=truth_table_clean[k,0])[0]
        b=np.where(tbin2<=truth_table_clean[k,1])[0]
        on_tm=np.intersect1d(a,b)
        for j in range(on_tm.shape[0]):
            truth_notes[on_tm[j],truth_table_clean[k,2].astype(np.int)]=1
    out_final = np.append(out_final,truth_notes,axis=0)
np.savetxt(out_path,inp_final,delimiter=',')
np.savetxt(out_path1,out_final,delimiter=',')
    

0.0
0.0005167958656330749
0.0010335917312661498
0.0015503875968992248
0.0020671834625322996
0.002583979328165375
0.0031007751937984496
0.0036175710594315244
0.004134366925064599
0.004651162790697674
0.00516795865633075
0.005684754521963824
0.006201550387596899
0.006718346253229974
0.007235142118863049
0.007751937984496124
0.008268733850129198
0.008785529715762274
0.009302325581395349
0.009819121447028423
0.0103359173126615
0.010852713178294573
0.011369509043927648
0.011886304909560724
0.012403100775193798
0.012919896640826873
0.013436692506459949
0.013953488372093023
0.014470284237726097
0.014987080103359173
0.015503875968992248
0.016020671834625324
0.016537467700258397
0.017054263565891473
0.01757105943152455
0.01808785529715762
0.018604651162790697
0.019121447028423774
0.019638242894056846
0.020155038759689922
0.020671834625323
0.02118863049095607
0.021705426356589147
0.022222222222222223
0.022739018087855296
0.023255813953488372
0.023772609819121448
0.02428940568475452
0.02480620155

0.2124031007751938
0.21291989664082686
0.21343669250645994
0.21395348837209302
0.2144702842377261
0.21498708010335918
0.21550387596899226
0.2160206718346253
0.2165374677002584
0.21705426356589147
0.21757105943152455
0.21808785529715763
0.2186046511627907
0.21912144702842376
0.21963824289405684
0.22015503875968992
0.220671834625323
0.22118863049095608
0.22170542635658916
0.2222222222222222
0.2227390180878553
0.22325581395348837
0.22377260981912145
0.22428940568475453
0.2248062015503876
0.22532299741602066
0.22583979328165374
0.22635658914728682
0.2268733850129199
0.22739018087855298
0.22790697674418606
0.2284237726098191
0.2289405684754522
0.22945736434108527
0.22997416020671835
0.23049095607235143
0.2310077519379845
0.23152454780361756
0.23204134366925064
0.23255813953488372
0.2330749354005168
0.23359173126614988
0.23410852713178296
0.234625322997416
0.2351421188630491
0.23565891472868217
0.23617571059431525
0.23669250645994833
0.2372093023255814
0.23772609819121446
0.23824289405684754

0.4310077519379845
0.4315245478036176
0.4320413436692506
0.4325581395348837
0.4330749354005168
0.43359173126614986
0.43410852713178294
0.434625322997416
0.4351421188630491
0.4356589147286822
0.43617571059431526
0.43669250645994834
0.4372093023255814
0.4377260981912145
0.4382428940568475
0.4387596899224806
0.4392764857881137
0.43979328165374676
0.44031007751937984
0.4408268733850129
0.441343669250646
0.4418604651162791
0.44237726098191216
0.44289405684754524
0.4434108527131783
0.4439276485788114
0.4444444444444444
0.4449612403100775
0.4454780361757106
0.44599483204134366
0.44651162790697674
0.4470284237726098
0.4475452196382429
0.448062015503876
0.44857881136950906
0.44909560723514214
0.4496124031007752
0.4501291989664083
0.4506459948320413
0.4511627906976744
0.4516795865633075
0.45219638242894056
0.45271317829457364
0.4532299741602067
0.4537467700258398
0.4542635658914729
0.45478036175710596
0.45529715762273903
0.4558139534883721
0.4563307493540052
0.4568475452196382
0.4573643410852713

0.6542635658914728
0.654780361757106
0.655297157622739
0.6558139534883721
0.6563307493540051
0.6568475452196383
0.6573643410852713
0.6578811369509044
0.6583979328165375
0.6589147286821705
0.6594315245478036
0.6599483204134367
0.6604651162790698
0.6609819121447028
0.661498708010336
0.662015503875969
0.6625322997416021
0.6630490956072351
0.6635658914728683
0.6640826873385013
0.6645994832041343
0.6651162790697674
0.6656330749354005
0.6661498708010336
0.6666666666666666
0.6671834625322998
0.6677002583979328
0.6682170542635659
0.668733850129199
0.6692506459948321
0.6697674418604651
0.6702842377260982
0.6708010335917313
0.6713178294573643
0.6718346253229974
0.6723514211886304
0.6728682170542636
0.6733850129198966
0.6739018087855297
0.6744186046511628
0.6749354005167959
0.6754521963824289
0.6759689922480621
0.6764857881136951
0.6770025839793282
0.6775193798449612
0.6780361757105943
0.6785529715762274
0.6790697674418604
0.6795865633074936
0.6801033591731266
0.6806201550387597
0.681136950904392

0.8790697674418605
0.8795865633074935
0.8801033591731267
0.8806201550387597
0.8811369509043928
0.8816537467700258
0.8821705426356589
0.882687338501292
0.883204134366925
0.8837209302325582
0.8842377260981912
0.8847545219638243
0.8852713178294573
0.8857881136950905
0.8863049095607235
0.8868217054263566
0.8873385012919897
0.8878552971576228
0.8883720930232558
0.8888888888888888
0.889405684754522
0.889922480620155
0.8904392764857881
0.8909560723514212
0.8914728682170543
0.8919896640826873
0.8925064599483205
0.8930232558139535
0.8935400516795866
0.8940568475452196
0.8945736434108527
0.8950904392764858
0.8956072351421188
0.896124031007752
0.896640826873385
0.8971576227390181
0.8976744186046511
0.8981912144702843
0.8987080103359173
0.8992248062015504
0.8997416020671835
0.9002583979328166
0.9007751937984496
0.9012919896640826
0.9018087855297158
0.9023255813953488
0.9028423772609819
0.903359173126615
0.9038759689922481
0.9043927648578811
0.9049095607235143
0.9054263565891473
0.9059431524547804


In [11]:
dir_path_in = os.path.join(os.getcwd(),'Data/Recordings/Testing')
out_path = os.path.join(os.getcwd(),'Data/Processed/Testing/input.txt')
out_path1 = os.path.join(os.getcwd(),'Data/Processed/Testing/tbin.txt')

files = listdir(dir_path_in)
inp_final = np.zeros((1,394))
out_final = np.zeros((1,88))
for i in range(len(files)):
    print(i/len(files))
    print(files[i])
    wav_path = os.path.join(os.getcwd(), 'Data/Recordings/Testing', files[i])
    fs, data = wavfile.read(wav_path)
    print(data.shape)
    if (data.shape[1]==2):
        data = data[:,0]
    spec1,freqbin1,tbin1 = stft(d=data,fs=fs,winlen=winlen1,nfft=nfft1)
    spec2,freqbin2,tbin2 = stft(d=data,fs=fs,winlen=winlen2,nfft=nfft2)
    
    Q1_temp = genData(spec=spec1,freqbin=freqbin1,tbin=tbin1)
    Q2 = genData(spec=spec2,freqbin=freqbin2,tbin=tbin2)

    Q1 = Q2.copy()
    
    
    for j in range(tbin2.shape[0]):
        near_t = np.argmin(np.abs(tbin2[j]-tbin1))
        Q1[j,:]=Q1_temp[near_t,:]
    
    inp_final = np.append(inp_final,np.concatenate((Q1,Q2),axis=1),axis=0)

np.savetxt(out_path1,tbin2,delimiter=',')
np.savetxt(out_path,inp_final,delimiter=',')

0.0
Viper's Drag.wav
(415251, 2)


In [8]:
p = os.path.join(os.getcwd(),'Data/Processed/Testing/tbin.txt')
a=pd.read_csv(p,sep=',',header=None)
b=np.zeros(1)
b
b=np.append(b,a)

In [9]:
b

array([0.        , 0.09287982, 0.26863946, 0.44439909, 0.62015873,
       0.79591837, 0.971678  , 1.14743764, 1.32319728, 1.49895692,
       1.67471655, 1.85047619, 2.02623583, 2.20199546, 2.3777551 ,
       2.55351474, 2.72927438, 2.90503401, 3.08079365, 3.25655329,
       3.43231293, 3.60807256, 3.7838322 , 3.95959184, 4.13535147,
       4.31111111, 4.48687075, 4.66263039, 4.83839002, 5.01414966,
       5.1899093 , 5.36566893, 5.54142857, 5.71718821, 5.89294785,
       6.06870748, 6.24446712, 6.42022676, 6.59598639, 6.77174603,
       6.94750567, 7.12326531, 7.29902494, 7.47478458, 7.65054422,
       7.82630385, 8.00206349, 8.17782313, 8.35358277, 8.5293424 ,
       8.70510204, 8.88086168, 9.05662132, 9.23238095])